In [1]:
import numpy as np
import pandas as pd

from sklearn.preprocessing import StandardScaler

# Preprocesar datos para importarlos a Redshift

Estandarizamos los nombres de las columnas para poder ser almacenadas en una base de datos relacional

In [2]:
df = pd.read_csv("data.csv")

In [3]:
df = df.rename(columns={ col: col.strip() for col in df.columns })
df.columns = df.columns.str.replace('[#,@,&,(,),?,¥,-,%,/]', '')
df.columns = df.columns.str.replace(' ', '_')
df.columns = df.columns.str.replace('-', '_')
df.columns

/tmp/ipykernel_2250/4271864808.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  df.columns = df.columns.str.replace('[#,@,&,(,),?,¥,-,%,/]', '')


Index(['Bankrupt', 'ROAC_before_interest_and_depreciation_before_interest',
       'ROAA_before_interest_and__after_tax',
       'ROAB_before_interest_and_depreciation_after_tax',
       'Operating_Gross_Margin', 'Realized_Sales_Gross_Margin',
       'Operating_Profit_Rate', 'Pre_tax_net_Interest_Rate',
       'After_tax_net_Interest_Rate',
       'Non_industry_income_and_expenditurerevenue',
       'Continuous_interest_rate_after_tax', 'Operating_Expense_Rate',
       'Research_and_development_expense_rate', 'Cash_flow_rate',
       'Interest_bearing_debt_interest_rate', 'Tax_rate_A',
       'Net_Value_Per_Share_B', 'Net_Value_Per_Share_A',
       'Net_Value_Per_Share_C', 'Persistent_EPS_in_the_Last_Four_Seasons',
       'Cash_Flow_Per_Share', 'Revenue_Per_Share_Yuan_',
       'Operating_Profit_Per_Share_Yuan_',
       'Per_Share_Net_profit_before_tax_Yuan_',
       'Realized_Sales_Gross_Profit_Growth_Rate',
       'Operating_Profit_Growth_Rate', 'After_tax_Net_Profit_Growth_Rate',
  

## Importar los datos desde Redshift

In [4]:
import os
import pandas as pd
from sqlalchemy import create_engine
import psycopg2

In [5]:
os.environ['REDSHIFT_USER'] = "awsuser"
os.environ['REDSHIFT_PASS'] = "Proyecto1*"
os.environ['REDSHIFT_ENDPOINT'] = "proyecto-integrador.cyzrunmtjnu8.us-east-1.redshift.amazonaws.com"
os.environ['REDSHIFT_PORT'] = "5439"
os.environ['REDSHIFT_DBNAME'] = "bankruptcy"
print('Informacion almacenada de manera exitosa')

Informacion almacenada de manera exitosa


In [6]:
# create sql engine to connect to redshift
engine_string = "postgresql+psycopg2://{}:{}@{}:{}/{}".format(
    os.getenv('REDSHIFT_USER'), os.getenv('REDSHIFT_PASS'), os.getenv('REDSHIFT_ENDPOINT'), 
    os.getenv('REDSHIFT_PORT'), os.getenv('REDSHIFT_DBNAME'))
engine = create_engine(engine_string)
print('Engine created successfully')

Engine created successfully


# Lectura del DataFrame

In [7]:
sql = """
    SELECT *
    FROM bankruptcy.public.dataset
"""
df = pd.read_sql(sql, engine)

In [8]:
df

,bankrupt,roac_before_interest_and_depreciation_before_interest,roaa_before_interest_and__after_tax,roab_before_interest_and_depreciation_after_tax,operating_gross_margin,realized_sales_gross_margin,operating_profit_rate,pre_tax_net_interest_rate,after_tax_net_interest_rate,non_industry_income_and_expenditurerevenue,...,net_income_to_total_assets,total_assets_to_gnp_price,no_credit_interval,gross_profit_to_sales,net_income_to_stockholders_equity,liability_to_equity,degree_of_financial_leverage_dfl,interest_coverage_ratio_interest_expense_to_ebit,net_income_flag,equity_to_liability
0,1,0.370594,0.424389,0.405750,0.601457,0.601457,0.998969,0.796887,0.808809,0.302646,...,0.716845,0.009219,0.622879,0.601453,0.827890,0.290202,0.026601,0.564050,1,0.016469
1,1,0.464291,0.538214,0.516730,0.610235,0.610235,0.998946,0.797380,0.809301,0.303556,...,0.795297,0.008323,0.623652,0.610237,0.839969,0.283846,0.264577,0.570175,1,0.020794
2,1,0.426071,0.499019,0.472295,0.601450,0.601364,0.998857,0.796403,0.808388,0.302035,...,0.774670,0.040003,0.623841,0.601449,0.836774,0.290189,0.026555,0.563706,1,0.016474
3,1,0.399844,0.451265,0.457733,0.583541,0.583541,0.998700,0.796967,0.808966,0.303350,...,0.739555,0.003252,0.622929,0.583538,0.834697,0.281721,0.026697,0.564663,1,0.023982
4,1,0.465022,0.538432,0.522298,0.598783,0.598783,0.998973,0.797366,0.809304,0.303475,...,0.795016,0.003878,0.623521,0.598782,0.839973,0.278514,0.024752,0.575617,1,0.035490
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6814,0,0.493687,0.539468,0.543230,0.604455,0.604462,0.998992,0.797409,0.809331,0.303510,...,0.799927,0.000466,0.623620,0.604455,0.840359,0.279606,0.027064,0.566193,1,0.029890
6815,0,0.475162,0.538269,0.524172,0.598308,0.598308,0.998992,0.797414,0.809327,0.303520,...,0.799748,0.001959,0.623931,0.598306,0.840306,0.278132,0.027009,0.566018,1,0.038284
6816,0,0.472725,0.533744,0.520638,0.610444,0.610213,0.998984,0.797401,0.809317,0.303512,...,0.797778,0.002840,0.624156,0.610441,0.840138,0.275789,0.026791,0.565158,1,0.097649
6817,0,0.506264,0.559911,0.554045,0.607850,0.607850,0.999074,0.797500,0.809399,0.303498,...,0.811808,0.002837,0.623957,0.607846,0.841084,0.277547,0.026822,0.565302,1,0.044009


# Validación de valores perdidos/nulos

In [9]:
np.unique((df.isnull().sum() >= 1).values)

array([False])

Debido a que no se encuentran valores perdidos/nulos, no es necesario evaluar que tratamiento se debe aplicar

# Guardar DataFrame

In [10]:
df.to_csv("/work/data/interim/000_ImportarRedshift.csv", index=False)

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=8ba77797-abb8-4bd4-bea9-bb2ef92463f4' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>